<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Vantage Analytic Library Demo 

</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Hypothesis Tests</b></p>

<p style = 'font-size:16px;font-family:Arial'>Statistical tests provide a means of testing whether the outcome of an experiment could have been accidental.  Numerous tests are provided in-database, all of the hypothesis testing variety.  These include the following 18 tests, categorized below, that are currently available in the Vantage Analytic Library XSP release 2.0.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Parametric Tests</b></p>

<p style = 'font-size:16px;font-family:Arial'>Two Sample T-Test for Equal Means</p>
    <li>T Paired</li>
    <li>T Unpaired</li>
    <li>T Unpaired with Indicator</li>
<p style = 'font-size:16px;font-family:Arial'>F-Test – N-Way</p>
<p style = 'font-size:16px;font-family:Arial'>F-Test/Analysis of Variance (Two Way Unequal Sample Size)</p>

<p style = 'font-size:16px;font-family:Arial'><b>Binomial Tests</b>

<li>Binomial/Ztest</li>
<li>Binomial Sign Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'><b>Tests based on Contingency</b>

<li>Chi Square Test</li>
<li>Median Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'><b>Kolmogorov/Smirnoff Tests</b>
    
<li>Kolmogorov/Smirnoff Test (One Sample)</li>
<li>Lilliefors Test</li>
<li>Shapiro-Wilk Test</li>
<li>D’Agostino and Pearson Test</li>
<li>Smirnov Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'><b>Rank Tests</b>

<li>Mann-Whitney/Kruskal-Wallis Test</li>
<li>Mann-Whitney/Kruskal-Wallis Independent Tests</li>
<li>Wilcoxon Signed Ranks Test</li>
<li>Frieman Test with Kendall’s Coefficient of Concordance & Spearman’s Rho</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'>In order to execute the Statistical Tests described in this book, the Statistical Test Metadata tables must be loaded into a database on the system to be analyzed.  These tables have been loaded into the TRNG_XSP database along with the other VAL XSP and Table Operators.  The Statistical Test functions all provide a parameter called statsdatabase that can be used to specify the database in which these tables are installed.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Vantage Analytic Library - Call Structure</b></p>

<p style = 'font-size:16px;font-family:Arial'>call ${XSPDB}.td_analyze('\<function name\>','\<database=database name\>;\<tablename=table name\>;\<columns=column1, column2\>,'\<param1\>=\<value1\>;\<param2\>=\<value2\>...');</p>

---

   
<p style = 'font-size:16px;font-family:Arial'>In the instructions below, use "SystemName=local" and "QLID=demo_user"</p>

In [1]:
%var SystemName=local

<p style = 'font-size:16px;font-family:Arial'>2) Next change QLID below to be equal to your QuickLook ID for Transcend or demo_user for DemoNow.  Your data lab will be the result database for all VAL calls where any output tables or views are created.</p>

In [2]:
%var QLID=demo_user

<p style = 'font-size:16px;font-family:Arial'>3) This next variable has been set for you to change, ONLY if you have installed the Vantage Analytic Library in some different database else keep it set to TRNG_XSP.  This is where the software, statistical test tables and demo data are installed.</p>

In [3]:
%var XSPDB=DEMO_Financial

In [4]:
%var VALDB=val

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>1. Start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [5]:
%connect local, hidewarnings=true

Password: ········


Success: 'local' connection established and activated for user 'demo_user', with default database 'demo_user'


<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [6]:
Set query_band='DEMO=VAL-Hypothesis_Tests.ipynb;' update for session;

Success: 1 rows affected

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>2. Getting Data for This Demo:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>


In [7]:
call get_data('DEMO_Financial_cloud'); ---takes about 35 seconds
---call get_data('DEMO_Financial_local'); ---takes about 100 seconds

Success: 0 rows affected

Success: 0 rows affected

,Message
1,That ran for 0:00:19.88 with 15 statements and 0 errors.


<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [8]:
call space_report();  -- optional, takes about 10 seconds

Success: 0 rows affected

Success: 0 rows affected

,Space_Report
1,"You have: #databases=1 #tables=40 #views=7 You have used 11.7 MB of 30,850.2 MB available - 0.0% ... Space Usage OK"
2,
3,Database Name #tables #views Avail MB Used MB
4,"demo_user 40 1 30,850.2 MB 11.7 MB"
5,DEMO_Financial 0 6 0.0 MB 0.0 MB


<p style = 'font-size:16px;font-family:Arial'>Change focus to the database specified by the variable "XSPDB" above.</p>

In [9]:
DATABASE ${XSPDB};

Success: 1 rows affected

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Demo data - Financial Customers/Accounts/Transactions</b></p>

<p style = 'font-size:16px;font-family:Arial'>The following data has been put into the ${XSPDB} database for the examples in the three different Jupyter Notebooks.  Its a simplistic ficticious dataset of banking customers (approx. 10K rows), Accounts (approx. 20K rows) and Transactions (approx. 1Million rows).  They are related to each other in the following ways:</p>

<img src="images/DemoData.png">

In [10]:
SELECT * FROM ${XSPDB}.Customer SAMPLE 10;

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,postal_code,state_code
1,29993326,37103.0,24,9,0,F,1,55113,MN
2,21800320,151780.2,49,5,5,F,4,90061,CA
3,23163724,.0,22,3,0,M,1,87194,NM
4,31340030,22084.6,79,9,0,M,2,96846,HI
5,25895442,3263.1,86,3,0,M,2,94162,CA
6,13626090,1929.0,80,9,1,F,2,96846,HI
7,14994067,23829.3,56,6,3,F,2,96849,HI
8,17718948,30356.3,47,4,4,F,2,19136,PA
9,31341134,104946.7,44,4,2,F,3,48208,MI
10,31337408,.0,13,2,0,M,1,77058,TX


<p style = 'font-size:16px;font-family:Arial'>Below command gives the metadata for the table.</p>

In [11]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2023.01.06_06.15.47.344_UTC
History ID:    376
Rows:          10 of 10
Parts:         2
Column Definitions:
    cust_id: INTEGER
    income: DECIMAL(15, 1)
    age: INTEGER
    years_with_bank: INTEGER
    nbr_children: INTEGER
    gender: VARCHAR(1)
    marital_status: VARCHAR(1)
    postal_code: VARCHAR(5)
    state_code: VARCHAR(2)


In [12]:
SELECT * FROM ${XSPDB}.Accounts SAMPLE 10;

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,starting_balance,ending_balance
1,000000001362928223,31347344,CK,Y,1995-01-15,147.140,93.800
2,000000001362571323,31339133,SV,Y,1991-02-23,2064.570,1831.090
3,456114321362815423,31344745,CC,Y,1995-10-27,11.000,1090.000
4,000000001363311311,14996421,SV,Y,1995-03-07,1209.768,1468.383
5,000000001363262220,27265240,CK,N,1992-07-28,3014.472,1736.163
6,000000001362500215,20437500,CK,Y,1995-12-04,2381.805,7497.714
7,456114321363259410,13632590,CC,Y,1988-03-11,723.159,1504.305
8,456114321362683418,24528294,CC,Y,1992-09-15,2925.670,5446.080
9,456114321362546423,31338558,CC,Y,1990-02-19,2351.610,1382.630
10,000000001363392212,16360704,CK,Y,1993-12-21,3268.452,184.149


In [13]:
SELECT * FROM ${XSPDB}.Transactions SAMPLE 10;

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
1,325,000000001363236313,-35.76,9.24,.000,35.76,1995-05-24,82148,V,TR
2,264,000000001362767212,-38.83,-87.37,1.000,38.83,1995-01-18,0,P,WD
3,460,000000001362993210,.00,.00,.000,3319.32,1995-06-19,151056,V,IQ
4,286,000000001362776222,-441.57,-432.57,.000,2035.27,1995-02-10,213849,C,TR
5,204,000000001363327217,-129.45,-53.45,.000,2260.90,1995-04-07,80119,K,TR
6,520,000000001362767210,138.49,1384.90,.000,760.44,1995-02-06,0,M,DP
7,36,456114321362850418,75.14,290.44,5.692,-107.18,1995-02-06,0,M,PM
8,360,456114321363345415,-68.99,-344.95,5.000,-2400.82,1995-06-17,80309,E,CG
9,1455,000000001362717215,.00,.00,5.000,208.33,1995-10-27,122152,C,IQ
10,60,000000001363145315,190.48,952.40,5.000,393.88,1995-03-12,91849,A,DP


---

<p style = 'font-size:16px;font-family:Arial'>The following Analytic Data Set (ADS) was created by joining all three tables above:</p>

In [14]:
CREATE TABLE ${QLID}.VAL_ADS AS (
    SELECT 
        T1.cust_id  AS cust_id
       ,MIN(T1.income) AS tot_income
       ,MIN(T1.age) AS tot_age
       ,MIN(T1.years_with_bank) AS tot_cust_years
       ,MIN(T1.nbr_children) AS tot_children
       ,CASE WHEN MIN(T1.marital_status) = 1 THEN 1 ELSE 0 END AS single_ind
       ,CASE WHEN MIN(T1.gender) = 'F' THEN 1 ELSE 0 END AS female_ind
       ,CASE WHEN MIN(T1.marital_status) = 2 THEN 1 ELSE 0 END AS married_ind
       ,CASE WHEN MIN(T1.marital_status) = 3 THEN 1 ELSE 0 END AS separated_ind
       ,MAX(CASE WHEN T1.state_code = 'CA' THEN 1 ELSE 0 END) AS ca_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'NY' THEN 1 ELSE 0 END) AS ny_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'TX' THEN 1 ELSE 0 END) AS tx_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'IL' THEN 1 ELSE 0 END) AS il_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'AZ' THEN 1 ELSE 0 END) AS az_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'OH' THEN 1 ELSE 0 END) AS oh_resident_ind
       ,MAX(CASE WHEN T2.acct_type = 'CK' THEN 1 ELSE 0 END) AS ck_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'SV' THEN 1 ELSE 0 END) AS sv_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'CC' THEN 1 ELSE 0 END) AS cc_acct_ind
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS ck_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS sv_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS cc_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS ck_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS sv_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS cc_avg_tran_amt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 1 THEN T3.tran_id ELSE NULL END) AS q1_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 2 THEN T3.tran_id ELSE NULL END) AS q2_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 3 THEN T3.tran_id ELSE NULL END) AS q3_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 4 THEN T3.tran_id ELSE NULL END) AS q4_trans_cnt
    FROM ${XSPDB}.Customer AS T1
        LEFT OUTER JOIN ${XSPDB}.Accounts AS T2
            ON T1.cust_id = T2.cust_id
        LEFT OUTER JOIN ${XSPDB}.Transactions AS T3
            ON T2.acct_nbr = T3.acct_nbr
GROUP BY T1.cust_id) WITH DATA UNIQUE PRIMARY INDEX (cust_id);

Success: 0 rows affected

In [15]:
SELECT * FROM ${QLID}.VAL_ADS SAMPLE 10;

,cust_id,tot_income,tot_age,tot_cust_years,tot_children,single_ind,female_ind,married_ind,separated_ind,ca_resident_ind,ny_resident_ind,tx_resident_ind,il_resident_ind,az_resident_ind,oh_resident_ind,ck_acct_ind,sv_acct_ind,cc_acct_ind,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
1,20437275,34035.0,30,6,2,1,1,0,0,0,1,0,0,0,0,1,0,1,462.2344736842105,0,2683.1267368421054,-1.90906432748538,0,-21.471988304093568,13,18,50,90
2,31339340,67534.9,36,6,3,0,1,0,0,1,0,0,0,0,0,1,1,1,321.86423076923074,98.70134615384616,473.0676923076923,-2.2194871794871793,-1.9578846153846154,-5.470512820512821,95,33,15,13
3,31354014,49785.8,49,6,0,1,1,0,0,1,0,0,0,0,0,1,0,1,1863.344057142857,0,796.4452571428571,-13.897428571428572,0,-2.2177142857142855,24,117,20,14
4,24539904,33238.8,33,1,4,0,0,0,1,0,0,0,0,0,0,1,1,1,618.358064516129,344.76354838709676,91.54935483870968,20.63827956989247,51.825806451612905,-23.17978494623656,12,14,27,40
5,20451915,34662.0,74,11,2,0,1,1,0,0,0,0,0,0,0,1,0,1,8193.686056179775,0,5533.940679775281,-51.40752808988764,0,1.7353932584269662,73,63,22,20
6,19077198,39124.4,40,9,1,1,1,0,0,1,0,0,0,0,0,1,0,1,294.8013375,0,0,0.38725,0,0,22,19,18,20
7,20438490,191772.0,59,6,3,0,0,1,0,1,0,0,0,0,0,1,0,1,31402.560527027028,0,4325.6806013513515,-580.3398648648649,0,168.44695945945946,115,18,8,7
8,19079718,31250.8,51,1,1,0,0,1,0,0,0,0,0,1,0,1,1,1,4310.0809714285715,442.0492857142857,515.4285714285714,-3.0162857142857145,0.7962142857142858,-1.5700714285714286,9,12,27,92
9,28622265,5964.4,75,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,13624960,.0,14,3,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1599.716,0,0,0.341,0,3,3,3,3


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Statistical Tests</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Binomial Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>
<p style = 'font-size:16px;font-family:Arial'>In a binomial test there are assumed to be n independent trials, each with two possible outcomes, each of equal probability.  You may choose to perform a binomial test, in which the sign of the difference between a first and second column is analyzed, or a sign test, in which the sign of a single column is analyzed.  In a binomial test, you may choose to use a probability different from the 0.5 default value, whereas in a sign test, the binomial probability is fixed at 0.5.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **binomialtest**</b></p>


<p style = 'font-size:16px;font-family:Arial'>The binomialtest parameter:</p>
<p style = 'font-size:16px;font-family:Arial'>
    <li>Is required</li>
    <li>Must be the first parameter</li>
    <li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The column to analyze. It must reside in the table indicated by the tablename parameter</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **binomialprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the binomial probability or allow it to default to 0.5.  This parameter may not be used for a sign test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **exactmatches**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This parameter determines what category to place exact matches in.  If zero, exact match is discarded.  If positive, match is placed with values greater than or equal to zero.  If negative or parameter not specified (default), match is placed with values less than or equal to zero.  Note that the exactmatches parameter is not allowed if this is a sign test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>A second column is required when performing a binomial test, and not allowed when performing a sign test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **singletail**</b></p>

<p style = 'font-size:16px;font-family:Arial'>A single-tailed test may be requested by setting this parameter to true.  A two-tailed test may be requested by setting this parameter to false or by not specifying it at all (the default value).  If the binomial probability is not 0.5, singletail must be set to true.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are binomial and sign, representing a binomial or sign test.  If not specified, the default test style is binomial.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected</p>

------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Binomial test producing an unpersisted result set with default values for thresholdprobabilty (.05), binomialprobability (0.5), singletail (false) and exactmatches (negative).</p>

In [16]:
call ${VALDB}.td_analyze('BinomialTest',
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          secondcolumn=ck_avg_bal;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,N,NPos,NNeg,BP,BinomialCallP_0.05
1,10458,6384,4074,2e-06,p


<p style = 'font-size:16px;font-family:Arial'>2.  Binomial test producing a result table and introducing a groupby variable gender with default values for other parameters.</p>

In [17]:
call ${VALDB}.td_analyze('BinomialTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          secondcolumn=ck_avg_bal;
                          groupby=female_ind;
                          outputdatabase=${QLID};
                          outputtablename=_val_binomial_example2;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

In [18]:
SELECT * FROM ${QLID}._val_binomial_example2;

,female_ind,N,NPos,NNeg,BP,BinomialCallP_0.05
1,1,5852,3717,2135,2e-06,p
2,0,4606,2667,1939,2e-06,p


<p style = 'font-size:16px;font-family:Arial'>3.  Sign test producing an unpersisted result set with default values for thresholdprobabilty (.05) and singletail (false).</p>

In [19]:
call ${VALDB}.td_analyze('BinomialTest',
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          teststyle=sign;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,N,NPos,NNeg,BP,BinomialCallP_0.05
1,10458,5894,4564,2e-06,p


<p style = 'font-size:16px;font-family:Arial'>4.  Sign test producing a result table and introducing a groupby variable gender with default values for other parameters.</p>

In [20]:
call ${VALDB}.td_analyze('BinomialTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          groupby=female_ind;
                          outputdatabase=${QLID};
                          outputtablename=_val_binomial_example4;
                          teststyle=sign;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

In [21]:
SELECT * FROM ${QLID}._val_binomial_example4;

,female_ind,N,NPos,NNeg,BP,BinomialCallP_0.05
1,1,5852,3514,2338,2e-06,p
2,0,4606,2380,2226,0.023268256,p


---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Kolmogorov-Smirnoff Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Kolmogorov-Smirnov Goodness-of-Fit Test is used to decide if a sample comes from a population with a specific distribution.  An attractive feature of this test is that the distribution of the K-S test statistic itself does not depend on the underlying cumulative distribution function being tested. Another advantage is that it is an exact test.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The numeric column that is tested to have a normal distribution. It must reside in the table indicated by the tablename parameter</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **kstest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The kstest parameter:
    <li>Is required</li>
    <li>Must be the first parameter</li>
    <li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Smirnov test, this parameter represents a categorical variable with two values that indicate the distribution to which the columnofinterest belongs. It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This parameter is not allowed when using the Smirnov test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>
    
<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are ks, l, sw, p and s, representing a Kolmogorov-Smirnov, Lilliefors, Shapiro-Wilk, D’Agostino and Pearson or Smirnov test.  If not specified, the default test style is Kolmogorov-Smirnov (ks).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected.</p>

--------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Kolmogorov-Smirnov test with group-by option.</p>

In [22]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=KS;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,tot_cust_years,Klm,M,KlmPValue,KlmPText,KlmCallP_5E-2
1,8,0.167601957,1014,0.01,<0.01,p
2,11,0.147381487,374,0.01,<0.01,p
3,12,0.145172899,182,0.01,<0.01,p
4,0,0.15574236700000005,176,0.01,<0.01,p
5,3,0.169747933,1160,0.01,<0.01,p
6,9,0.156509661,845,0.01,<0.01,p
7,4,0.17757486300000003,1137,0.01,<0.01,p
8,14,0.11384552699999997,39,0.2,>0.20,a
9,1,0.168141482,526,0.01,<0.01,p
10,10,0.186458308,613,0.01,<0.01,p


<p style = 'font-size:16px;font-family:Arial'>2.  Lilliefors test with group-by option.</p>

In [23]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=L;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,tot_cust_years,Lilliefors,M,LillieforsPValue,LillieforsPText,LillieforsCallP_5E-2
1,8,0.16437102548323468,1014,0.01,<0.01,p
2,11,0.14539964207486628,374,0.01,<0.01,p
3,12,0.14397700704395605,182,0.01,<0.01,p
4,0,0.15455723972727275,176,0.01,<0.01,p
5,3,0.1652836109310345,1160,0.01,<0.01,p
6,9,0.15286021289940832,845,0.01,<0.01,p
7,4,0.17220877782761657,1137,0.01,<0.01,p
8,14,0.11152678682051276,39,0.2,>0.20,a
9,1,0.16513402253992393,526,0.01,<0.01,p
10,10,0.1838866707161501,613,0.01,<0.01,p


<p style = 'font-size:16px;font-family:Arial'>3.  Shapiro-Wilk test with group-by option</p>

In [24]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=SW;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,tot_cust_years,Shw,N,ShapiroWilkPValue,ShapiroWilkPText,ShapiroWilkCallP_5E-2
1,2,0.7380468157991138,888,1e-06,,p
2,13,0.8639321777575529,45,0.01,<0.01,p
3,7,0.7192490933650109,1118,1e-06,,p
4,14,0.9308345215745261,39,0.029586303936315124,,p
5,1,0.7768408108631809,526,1e-06,,p
6,9,0.7646871828003444,845,1e-06,,p
7,12,0.8145813511010318,182,1e-06,,p
8,0,0.8439237666531059,176,1e-06,,p
9,8,0.7397561124692311,1014,1e-06,,p
10,10,0.7328519330437476,613,1e-06,,p


<p style = 'font-size:16px;font-family:Arial'>4.  D'Agostino and Pearson Test with group-by option.</p>

In [25]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=P;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,tot_cust_years,T,Zkurtosis,Zskew,ChiPValue,ChiPText,ChiCallP_5E-2
1,5,670.6249123670651,13.339258887906924,22.19660074625079,0.0001,<0.0001,p
2,13,6.550323087619043,-0.2341478075898076,2.548626667799725,0.039921993675254674,,p
3,7,688.1595653640863,13.841639526324515,22.28381880619606,0.0001,<0.0001,p
4,0,56.45743848242036,3.7488191061924487,6.511819545370311,0.0001,<0.0001,p
5,3,589.3437122939496,12.388685407891911,20.87736061283216,0.0001,<0.0001,p
6,11,147.96721301835333,6.177449794326851,10.478851418782714,0.0001,<0.0001,p
7,12,56.863309905678044,3.6355690358930906,6.606507980085502,0.0001,<0.0001,p
8,14,3.328860179628914,-0.6811192150956611,1.6926124170808818,0.20446845179761405,,a
9,1,244.19995139460286,8.296957017858563,13.242373489613197,0.0001,<0.0001,p
10,9,439.9169174138988,11.233736234248122,17.71214520133684,0.0001,<0.0001,p


<p style = 'font-size:16px;font-family:Arial'>5.  Smirnov test with group-by option.</p>

In [26]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          columns=female_ind;
                          teststyle=S;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,tot_cust_years,M,N,D,SmirnovPValue,SmirnovPText,SmirnovCallP_5E-2
1,8,450,564,1.3487542965938886,0.005,<0.005,p
2,11,158,216,1.285839772469803,0.005,<0.005,p
3,12,77,105,1.342031698510656,0.005,<0.005,p
4,0,74,102,1.313851469476118,0.005,<0.005,p
5,3,495,665,1.3747929978080613,0.005,<0.005,p
6,9,374,471,1.3686535531466877,0.005,<0.005,p
7,4,503,634,1.376370284874307,0.005,<0.005,p
8,14,16,23,1.3716711154322194,0.000101,,p
9,1,226,300,1.3719868864800215,0.005,<0.005,p
10,10,263,350,1.3713273499381309,0.005,<0.005,p


<p style = 'font-size:16px;font-family:Arial'>6.  Kolmogorov-Smirnov test that stores its results in an output table val_ks_out1 and does not contain a group-by option.

In [27]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=KS; 
                          statsdatabase=${VALDB};
                          outputdatabase=${QLID};
                          outputtablename=_val_ks_out1');

Success: 0 rows affected

In [28]:
SELECT * FROM ${QLID}._val_ks_out1;

,Klm,M,KlmPValue,KlmPText,KlmCallP_5E-2
1,0.15818501100000001,10458,0.01,<0.01,p


---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Parametric Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Parametric tests make assumptions about the data, such as the observations being normally distributed.  This can be verified with a test of normality prior to executing a parametric test.  Both T-Tests and F-Tests are provided.  T-Tests may be either paired or unpaired, while the unpaired T-Tests may be with or without an indicator variable.</p>  

<p style = 'font-size:16px;font-family:Arial'>F-Tests may be 1-way, 2-way or 3-way.  2-way tests may have equal or unequal cell counts (distinct column values), while the 3-way test must have equal cell counts.  A 1-way test has 1 independent input column, a 2-way test has 2 independent columns and a 3-way test has 3 independent columns in addition to a dependent “column of interest”.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **parametrictest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The parametrictest parameter:
    <li>Is required</li>
    <li>Must be the first parameter</li>
    <li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The column representing the dependent variable in an F-Test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The columns representing independent variables to be analyzed in a F-Test N-Way with Equal Cell Counts analysis.  There may be 1, 2 or 3 columns listed in this parameter.  If 2 or 3 columns, the number of distinct values for each combination of column values should be the same.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **equalvariance**</b></p>

<p style = 'font-size:16px;font-family:Arial'>An option available with the T-Test where, if true, the variance of the two samples (columns) is assumed to be equal.  The default assumption is that the variances are not equal.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>For a T-Test, the column representing the first variable to analyze. It must reside in the table indicated by the tablename parameter.  For an F-Test, the column representing the first independent variable in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumnvalues**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is a required option for a 2-way F-Test with Unequal Cell Counts.  It is a list of the firstcolumn values to be included in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>For a T-Test analysis, when gensqlonly is true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This option is not available for an F-Test analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **paired**</b></p>

<p style = 'font-size:16px;font-family:Arial'>An option of the T-Test, which when set to true, indicates that the first and second column values are matched with each other and the mean difference is analyzed.  The paired option defaults to false.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>For a T-Test, the column representing the second variable to analyze. It must reside in the table indicated by the tablename parameter.  If the withindicator option is set to true, the second column is used to define two analysis categories, one where the second column is negative or zero, and another where the second column is positive.  For an F-Test, the column representing the second independent variable in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumnvalues**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is a required option for a 2-way F-Test with Unequal Cell Counts.  It is a list of the secondcolumn values to be included in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are t, fnway and f2way, as outlined below. The default is t for T-Test.
    
<li>T-Test paired, unpaired or unpaired with indicator variable (second column)</li>
<li>F-Test N-Way with Equal Cell Counts (1, 2 or 3 columns with same number of distinct values)</li>
<li>F-Test 2-Way with Unequal Cell Counts (2 columns with possibly different numbers of distinct values)</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **withindicator**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This option may be used with an un-paired T-Test, that is when teststyle=t and paired=false.  If the withindicator option is set to true, the second column is used to indicate there are two analysis categories, one for the case where the second column is negative or zero, and another when the second column is positive.</p>

--------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  T-Test producing an unpersisted result set with default values for teststyle (t) and  thresholdprobabilty (.05). </p> 

In [29]:
call ${VALDB}.td_analyze('ParametricTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=cc_avg_bal;
                          secondcolumn=sv_avg_bal;
                          paired=true;
                          equalvariance=true;
                          groupby=tot_age,female_ind;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,tot_age,female_ind,D_F,TTestPValue,T,TTestCallP_0.05
1,45,0,68,0.0001292,-4.059830394523732,n
2,93,0,3,0,,
3,94,1,7,0.1132148,1.8135209270199906,a
4,90,1,9,0.1555528,1.545231261242011,a
5,10,1,5,0.1100709,1.9429580983579042,a
6,45,1,104,0.0438849,-2.0436408668962054,n
7,49,1,95,0.0001084,-4.035018616442392,n
8,77,1,40,0.0221685,-2.384227212864992,n
9,61,1,86,0.0250818,2.2760531913263917,p
10,75,0,27,0.1025436,1.6862479715409269,a


<p style = 'font-size:16px;font-family:Arial'>2.  One-Way F-Test producing an unpersisted result set.</p>

In [30]:
call ${VALDB}.td_analyze('ParametricTest',
                         'teststyle=fnway;
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          columns=years_with_bank;
                          groupby=gender;
                          thresholdprobability=0.01;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,gender,DF,DFErr,F,FPValue,FPText,FCallP_0.01
1,F,14,5837,7.3394608434061785,0.001,<0.001,p
2,M,14,4591,4.946361887968362,0.001,<0.001,p


<p style = 'font-size:16px;font-family:Arial'>3.  2-way F-Test with Unequal Cell Counts producing an unpersisted result set.</p>

In [31]:
call ${VALDB}.td_analyze('ParametricTest',
                         'teststyle=f2way;
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          firstcolumn=years_with_bank;
                          secondcolumn=gender;
                          thresholdprobability=0.05;
                          firstcolumnvalues=0,1,2,3,4,5,6,7;
                          secondcolumnvalues=M,F;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

,DF,Fmodel,DFErr,DF_1,F1,DF_2,F2,DF_12,F12,Fmodel_PValue,Fmodel_PText,Fmodel_CallP_0.05,F1_PValue,F1_PText,F1_CallP_0.05,F2_PValue,F2_PText,F2_CallP_0.05,F12_PValue,F12_PText,F12_CallP_0.05
1,15,9.710997531996655,7330,7,11.561359030335888,1,51.83852316974105,7,1.8424180854082204,0.001,<0.001,p,0.001,<0.001,p,0.001,<0.001,p,0.07854148392066758,,a


---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Rank Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Statistical tests of this type calculate statistics based on the rank of variables rather than variable values.  In general, data that are ranked and ordinal may be analyzed by these tests.  Within some restraints, either numeric or non-numeric data may be analyzed.  Supported rank tests include the following:

<li>Mann-Whitney/Kruskal-Wallis Test</li>
<li>Wilcoxon Signed Ranks Test</li>
<li>Friedman Test with Kendall’s Coefficient of Concordance & Spearmans’ Rho</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'>The choice between the Mann-Whitney and Kruskal-Wallis tests is made automatically, looking at the number of distinct values of the independent variable.  A variation of the Mann-Whitney test considers each requested variable individually, rather than combined, performing a series of independent tests.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **ranktest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The ranktest parameter:
<li>Is required</li>
<li>Must be the first parameter</li>
<li>Is always enclosed in single quotes</li>
</p>    

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **blockcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Friedman test, the column that represents blocks. They must reside in the table indicated by the tablename parameter.  Note that when pairing treatment and block column values, a division by zero error may occur if unequal cell counts are found.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney and Friedman tests, the column specified with this parameter indicates the dependent variable. If non-numeric, it will be ranked alphanumerically.  It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney test, the columns specified with this parameter represent the independent variables. They must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Wilcoxon test, the column that represents the first sample variable. It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This parameter is not allowed when using the Mann-Whitney test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **includezero**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Wilcoxon test.  Ordinarily, the Wilcoxon test discards cases with zero differences.  This option, when set to true, includes these cases with the positive count.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **independent**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney test, its selection by setting independent=true, indicates that a variation of the Mann-Whitney test should be performed, considering each requested variable individually, rather than in combination, performing a series of independent tests.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Wilcoxon test, the column that represents the second sample variable. It must reside in the table indicated by the tablename parameter.  Note that when pairing treatment and block column values, a division by zero error may occur if unequal cell counts are found.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **singletail**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney and Wilcoxon tests.  If the Mann-Whitney test becomes a Kruskall-Wallis test, the singletail option is invalid.  By default, a two-tailed test is performed.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are mw, friedman and wilcoxon, representing a Mann-Whitney, Friedman or Wilcoxon test.  If not specified, the default test style is Mann-Whitney (mw).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this "alpha" probability the null hypothesis is rejected.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **treatmentcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Friedman test, the column that represents the independent categorical variable. They must reside in the table indicated by the tablename parameter.</p>

--------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Mann-Whitney test with a threshold probability of 0.01.</p>

In [32]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=mw;
                          statsdatabase=${VALDB};
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          columns=gender;
                          groupby=years_with_bank;
                          thresholdprobability=0.01;');

Success: 0 rows affected

,years_with_bank,Z,MannWhitneyPValue,MannWhitneyCallP_1E-2
1,12,-1.5058426157251128,0.132067226,a
2,3,-1.2277790991000173,0.219447096,a
3,4,-2.1776290109979204,0.029405932,a
4,2,0.2632551342578662,0.792550692,a
5,11,-1.329063927064864,0.183848096,a
6,10,-3.2379133262969324,0.001203832,n
7,7,-3.9525279885398006,7.7211e-05,n
8,13,-0.8617464449585489,0.388687424,a
9,14,-0.49966021063014066,0.617075066,a
10,8,-3.508754308300577,0.00044988199999999997,n


<p style = 'font-size:16px;font-family:Arial'>2.  A set of Mann-Whitney independent tests.  The threshold probability assumes the default value of 0.05.</p>

In [33]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=mw;
                          statsdatabase=${VALDB};
                          independent=true;
                          database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          columns=female_ind,cc_acct_ind,ck_acct_ind,sv_acct_ind; ');

Success: 0 rows affected

,_twm_independent_variable,Z,MannWhitneyPValue,MannWhitneyCallP_5E-2
1,female_ind,7.771020590981125,6e-07,p
2,sv_acct_ind,9.377586778932272,6e-07,p
3,cc_acct_ind,-19.115808047105958,6e-07,n
4,ck_acct_ind,-16.06418788422782,6e-07,n


<p style = 'font-size:16px;font-family:Arial'>3.  Wilcoxon Test - the threshold probability assumes the default value of 0.05.</p>

In [34]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=wilcoxon;
                          statsdatabase=${VALDB};
                          database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=ck_avg_bal;
                          secondcolumn=sv_avg_bal;
                          groupby=tot_cust_years; ');

Success: 0 rows affected

,tot_cust_years,N,Z_,WilcoxonPValue,WilcoxonCallP_5E-2
1,11,315,-8.214871312499245,6e-07,n
2,13,39,-2.6793596002730693,0.007384332,n
3,7,918,-11.640686617914307,6e-07,n
4,5,1010,-16.95474369795407,6e-07,n
5,9,681,-9.077457386798521,6e-07,n
6,4,977,-18.779568835263486,6e-07,n
7,0,150,-4.790039389110829,1.669944e-06,n
8,14,33,-2.206676899498888,0.027313964,n
9,3,1003,-17.354268985435144,6e-07,n
10,1,454,-8.994275181286014,6e-07,n


<p style = 'font-size:16px;font-family:Arial'>4.  A Friedman Test using a specially prepared input table called VAL_Friedman_WorkTable.  To build the VALFriedmanWorkTable, the following SQL may be used.  Note that the value ‘18’ is the smallest count of value combinations in the gender and marital_status columns in the Customer table.  The value '18'  is determined using the following query:</p>

In [35]:
SELECT 
     MIN("T0"."_val_N") AS "smallest_count"
FROM 
( SELECT 
     "S0"."marital_status" AS "marital_status"
    ,"S0"."gender" AS "gender"
    ,COUNT(*) AS "_val_N"
FROM "${XSPDB}"."Customer" AS "S0"
GROUP BY "S0"."marital_status", "S0"."gender"
) AS "T0"
;

,smallest_count
1,252


In [36]:
CREATE SET TABLE "${QLID}"."VAL_Friedman_Worktable" AS (
SELECT
    "cust_id"
    ,"gender"
    ,"marital_status"
    ,"income"
    ,"ckacct"
    ,"svacct"
    ,SAMPLEID AS "xSampleId"
FROM "DEMO_Financial"."Customer_Analysis"
SAMPLE 
    WHEN gender='f' and marital_status='1' THEN 18
    WHEN gender='f' and marital_status='2' THEN 18
    WHEN gender='f' and marital_status='3' THEN 18
    WHEN gender='f' and marital_status='4' THEN 18
    WHEN gender='m' and marital_status='1' THEN 18
    WHEN gender='m' and marital_status='2' THEN 18
    WHEN gender='m' and marital_status='3' THEN 18
    WHEN gender='m' and marital_status='4' THEN 18
END
) WITH DATA PRIMARY INDEX("cust_id");

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3802:Database 'TRNG_XSP' does not exist.

In [37]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=friedman;
                          statsdatabase=${VALDB};
                          database=${QLID};
                          tablename=VAL_Friedman_Worktable;
                          columnofinterest=income;
                          treatmentcolumn=gender;
                          blockcolumn=marital_status;');

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:[SQLState 42S02] Object 'demo_user.VAL_Friedman_Worktable' does not exist.

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Chi Square Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Statistical tests of this type are based on a matrix of frequencies or counts.  A frequency pattern that is non-random is sought in the matrix.  Supported tests of this type include the following:

<li>Chi Square Test</li>
</p>

<p style = 'font-size:16px;font-family:Arial'>Besides a Chi Square value, other measures are computed in a Chi Square Test, including a Phi Coefficient,  Cramer’s V, Likelihood Ratio Chi Square, Continuity-Adjusted Chi Square and Contingency Coefficient.

<li>Median Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'>A Median Test is a variation of Chi Square Test wherein samples are tested to see if their populations have the same median value.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **chisquaretest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The chisquaretest parameter:
    
<li>Is required</li>
<li>Must be the first parameter</li>
<li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Median Test, the column specified with this parameter indicates the dependent variable (numeric).  It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Median Test, the columns specified with this parameter represent the independent variables (categorical). They must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Chi Square Test, one or more columns representing the first of variable pairs for analysis. They must reside in the table indicated by the tablename parameter.  Note that the number of combinations of firstcolumns and secondcolumns may not exceed 100.  Note also that if the product of the number distinct values in these column pairs exceeds 2000, the analysis of that combination is skipped (but not if gensqlonly is true).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Available only in the Median Test, one or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Chi Square Test, one or more columns representing the second of variable pairs for analysis. They must reside in the table indicated by the tablename parameter.  Note that the number of combinations of firstcolumns and secondcolumns may not exceed 100.  Note also that if the product of the number distinct values in these column pairs exceeds 2000, the analysis of that combination is skipped (but not if gensqlonly is true).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are chisq and median, representing a Chi Square or Median test.  If not specified, the default test style is Chi Square (chisq).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this "alpha" probability the null hypothesis is rejected.</p>

---------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Chi Square Test with non-persisted results.</p>

In [38]:
call ${VALDB}.td_analyze('chisquaretest',
                         'statsdatabase=${VALDB};
                          teststyle=chisq;
                          database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumns=female_ind,single_ind;
                          secondcolumns=sv_acct_ind,cc_acct_ind,ck_acct_ind;');

Success: 0 rows affected

,column1,column2,Chisq,DF,Z,CramersV,PhiCoeff,LlhChiSq,ContAdjChiSq,ContinCoeff,ChiPValue,ChiPText,ChiCallP_5E-2
1,female_ind,ck_acct_ind,51.96329114361818,1,6.266072413236709,0.07048943119690093,0.07048943119690093,51.79027403137195,51.65502985345838,0.07031495858282666,0.0001,<0.0001,p
2,single_ind,sv_acct_ind,111.33800714811451,1,8.555265794438732,0.10318044653217087,0.10318044653217087,112.15138339109859,110.90737820419025,0.10263555340590276,0.0001,<0.0001,p
3,female_ind,sv_acct_ind,73.52013882165667,1,7.236808261366889,0.08384532268145403,0.08384532268145403,73.48997707707434,73.17998066361037,0.08355214963436527,0.0001,<0.0001,p
4,single_ind,cc_acct_ind,21.541917084755497,1,4.252562129760672,0.04538557648792909,0.04538557648792909,21.461624947837834,21.347967794596418,0.04533890482563551,0.0001,<0.0001,p
5,single_ind,ck_acct_ind,3.722261430607426,1,1.6376557333933495,0.018865969038997184,0.018865969038997184,3.7119507979566606,3.6377618158186986,0.018862612501963423,0.055246604425102215,,a
6,female_ind,cc_acct_ind,84.45466569698048,1,7.657179013939365,0.08986436396365856,0.08986436396365856,84.2873938591323,84.08086832585147,0.08950369247502081,0.0001,<0.0001,p


<p style = 'font-size:16px;font-family:Arial'>2.  Chi Square Test with results persisted in an output table.</p>

In [39]:
call ${VALDB}.td_analyze('chisquaretest',
                         'statsdatabase=${VALDB};
                          teststyle=chisq;
                          database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumns=female_ind,single_ind;
                          secondcolumns=sv_acct_ind,cc_acct_ind,ck_acct_ind;
                          outputdatabase=${QLID};
                          outputtablename=_val_chisquare_out;');

Success: 0 rows affected

In [40]:
SELECT * FROM ${QLID}._val_chisquare_out ORDER BY 1;

,column1,column2,Chisq,DF,Z,CramersV,PhiCoeff,LlhChiSq,ContAdjChiSq,ContinCoeff,ChiPValue,ChiPText,ChiCallP_5E-2
1,female_ind,cc_acct_ind,84.45466569698048,1,7.657179013939365,0.08986436396365856,0.08986436396365856,84.28739385913224,84.08086832585147,0.08950369247502081,0.0001,<0.0001,p
2,female_ind,sv_acct_ind,73.52013882165669,1,7.23680826136689,0.08384532268145405,0.08384532268145405,73.48997707707434,73.17998066361037,0.08355214963436529,0.0001,<0.0001,p
3,female_ind,ck_acct_ind,51.963291143618186,1,6.266072413236709,0.07048943119690093,0.07048943119690093,51.79027403137195,51.65502985345838,0.07031495858282666,0.0001,<0.0001,p
4,single_ind,sv_acct_ind,111.33800714811451,1,8.555265794438732,0.10318044653217087,0.10318044653217087,112.15138339109859,110.90737820419025,0.10263555340590276,0.0001,<0.0001,p
5,single_ind,cc_acct_ind,21.5419170847555,1,4.252562129760672,0.04538557648792909,0.04538557648792909,21.461624947837834,21.347967794596414,0.04533890482563551,0.0001,<0.0001,p
6,single_ind,ck_acct_ind,3.7222614306074258,1,1.6376557333933492,0.01886596903899718,0.01886596903899718,3.7119507979566606,3.6377618158186986,0.01886261250196342,0.05524660442510223,,a


<p style = 'font-size:16px;font-family:Arial'>3.  Median Test with groupby column years_with_bank, thresholdprobability set to 0.01 and results returned in a result set.</p> 

In [41]:
call ${VALDB}.td_analyze('chisquaretest', 
                         'statsdatabase=${VALDB};
                          teststyle=median;
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          columns=marital_status;
                          groupby=years_with_bank;
                          thresholdprobability=0.01;');

Success: 0 rows affected

,years_with_bank,ChiSq,DF,MedianPValue,MedianPText,MedianCallP_1E-2
1,6,67.65712906289531,3,0.0001,<0.0001,p
2,10,20.81869569607782,3,0.00013383716364114627,,p
3,13,2.8559997422237497,3,0.25,>0.25,a
4,8,64.78631020023145,3,0.0001,<0.0001,p
5,1,54.205555555555556,3,0.0001,<0.0001,p
6,2,111.94131226907886,3,0.0001,<0.0001,p
7,5,98.98244733043795,3,0.0001,<0.0001,p
8,9,23.73665898145068,3,0.0001,<0.0001,p
9,0,16.604904354904356,3,0.0008843323011702343,,p
10,4,77.54838819017066,3,0.0001,<0.0001,p


---

<p style = 'font-size:16px;font-family:Arial'>Since we are in a beta phase, the following command can be used to determine the specific release you are running:</p>

In [42]:
call ${VALDB}.td_analyze('version','');

Success: 0 rows affected

,version
1,VAL In-DB 2.1.0.0


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>15. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:18px;font-family:Arial'>Clean up work tables to prevent error next time.</p>

In [43]:
DROP TABLE "${QLID}"."VAL_Friedman_Worktable";

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'demo_user.VAL_Friedman_Worktable' does not exist.

In [44]:
DROP TABLE "${QLID}"."VAL_ADS";

Success: 42 rows affected

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:18px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [45]:
Database ${QLID};

Success: 1 rows affected

In [46]:
call remove_data('DEMO_Financial');

Success: 0 rows affected

,Message
1,Removed objects related to DEMO_Financial. That ran for 0:00:05.45


<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>